In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
#         img_arr = img_arr / 255.0
        images.append(img_arr)
    X = np.column_stack(images)
#     print(images.shape)
#     print(X.shape)
    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
data_root_path = '/home/ec2-user/Neural_Network/Neural_Network/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test').T

print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs].T
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1).T
y_train = np.delete(y_train,indexs)

{'horse': 7, 'airplane': 0, 'automobile': 1, 'cat': 3, 'bird': 2, 'truck': 9, 'frog': 6, 'ship': 8, 'deer': 4, 'dog': 5}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
print(X_validation.shape)
print(X_test.shape)
arr = np.arange(X_train.shape[0])
np.random.shuffle(arr)
print(arr)

(5000, 3072)
(10000, 3072)
[24106 26248  3012 ..., 26610 37087 38495]


In [6]:
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        m = X.shape[0]
        start_index = np.random.randint(0, m - batch_size)
        X_batch = X[start_index:(start_index + batch_size), :]
        y_batch = y[start_index:(start_index + batch_size)]
#         X_batch = np.reshape(X_batch, [batch_size, 32, 32, 3])
#         for i in range(batch_size):
#             get_distorted_image(X_batch[i])
        return X_batch, y_batch
    
def get_distorted_image(image):
#     X_batch = np.reshape(X_batch, [batch_size, 32, 32, 3])
#     for i in range(batch_size):
#         get_distorted_image(X_batch[i])
    d_img = tf.image.random_flip_left_right(image)
    d_img = tf.image.random_brightness(d_img, max_delta=63)
    d_img = tf.image.random_contrast(d_img, lower=0.2, upper=0.8)
    d_img = tf.image.per_image_standardization(d_img)
    
    return d_img
#     image = image.eval()
#     d_img = d_img.eval()
#     plt.imshow(image)
#     plt.figure()
#     plt.imshow(d_img)

In [7]:
logdir = 'tf_logs/summary_resNet4'
iteration = 15000
batch_size = 128
learning_rate = 0.001

In [8]:
# try use a function to create a layer
# def create_conv(filters, input_layer):
#     conv = tf.layers.conv2d(inputs=input_layer, filters=filters, kernel_size=3, strides=1, 
#                                     padding = 'SAME', activation = tf.nn.relu, name='conv')
#     pool = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool')
#     return pool

In [8]:
def save_variable(var, name):
    with tf.name_scope(name) as scope:
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        stddev = tf.sqrt(tf.reduce_mean(var - mean))
        tf.summary.scalar('stddev', stddev)
        tf.summary.histogram('histgram', var)

In [9]:
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
with tf.device('/gpu:0'):
    with tf.name_scope('input') as scope:
        X = tf.placeholder(tf.float32, shape = (None, 3072))
        y = tf.placeholder(tf.int32, shape = (None))
        dropout_rate = tf.placeholder(tf.float32, shape=())
        training = tf.placeholder(tf.bool)
        input_layer = tf.reshape(X, [-1, 32, 32, 3])
#         input_layer = tf.reshape(X, [32, 32, 3])
        result = tf.map_fn(lambda img: get_distorted_image(img), input_layer)
#         distorted = get_distorted_image(X)

    with tf.name_scope('conv1') as scope:
        conv1 = tf.layers.conv2d(inputs=result, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv1',
                                kernel_regularizer = regularizer)
#         weight1 = tf.contrib.layers.flatten(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1'))
        weight1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1')[0]
        batch_norm1 = tf.layers.batch_normalization(conv1, axis = 1, name='batch_norm1')
        dropout1 = tf.layers.dropout(batch_norm1,rate=0.2, training=training, name='dropout1')
#         pool1 = tf.layers.max_pooling2d(inputs=batch_norm1, pool_size=2, strides=2, padding='SAME', name='pool1')

    with tf.name_scope('conv2') as scope:
        conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv2',
                                kernel_regularizer = regularizer)
        weight2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv2')[0]
        
        batch_norm2 = tf.layers.batch_normalization(conv2, axis = 1, name='batch_norm2')
        combine2 = tf.add(batch_norm1, batch_norm2, name='combine2')
        dropout2 = tf.layers.dropout(combine2,rate=0.3, training=training, name='dropout2')
        
#         pool2 = tf.layers.max_pooling2d(inputs=batch_norm2, pool_size=2, strides=2, padding='SAME', name='pool2')
        
    with tf.name_scope('conv3') as scope:
        conv3 = tf.layers.conv2d(inputs=dropout2, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv3',
                                kernel_regularizer = regularizer)
        weight3 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv3')[0]
        
        batch_norm3 = tf.layers.batch_normalization(conv3, axis = 1, name='batch_norm3')
        combine3 = tf.add(batch_norm3, combine2, name='combine3')
#         dropout3 = tf.layers.dropout(combine3, rate=0.4, training=training, name='dropout3')
        pool3 = tf.layers.max_pooling2d(inputs=combine3, pool_size=2, strides=2, padding='SAME', name='pool3')
        
    with tf.name_scope('conv4') as scope:
        conv4 = tf.layers.conv2d(inputs=pool3, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv4',
                                kernel_regularizer = regularizer)
        weight4= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv4')[0]
        
        batch_norm4 = tf.layers.batch_normalization(conv4, axis = 1, name='batch_norm4')
        combine4 = tf.add(pool3, batch_norm4, name='combine4')
        dropout4 = tf.layers.dropout(combine4,rate=0.3, training=training, name='dropout4')
#         pool4 = tf.layers.max_pooling2d(inputs=batch_norm4, pool_size=2, strides=2, padding='SAME', name='pool4')

    with tf.name_scope('conv5') as scope:
        conv5 = tf.layers.conv2d(inputs=dropout4, filters=128, kernel_size=3, strides=2, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv5',
                                kernel_regularizer = regularizer)
        weight5= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv5')[0]
        
        batch_norm5 = tf.layers.batch_normalization(conv5, axis = 1, name='batch_norm5')
        dropout5 = tf.layers.dropout(batch_norm5,rate=0.3, training=training, name='dropout5')
        
    with tf.name_scope('conv6') as scope:
        conv6 = tf.layers.conv2d(inputs=dropout5, filters=128, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv6',
                                kernel_regularizer = regularizer)
        weight6= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv6')[0]
        
        batch_norm6 = tf.layers.batch_normalization(conv6, axis = 1, name='batch_norm6')
        combine6 = tf.add(batch_norm5, batch_norm6, name='combine6')
#         dropout5 = tf.layers.dropout(batch_norm5,rate=0.3, training=training, name='dropout5')
        pool6 = tf.layers.max_pooling2d(inputs=combine6, pool_size=2, strides=2, padding='SAME', name='pool6')

    with tf.name_scope('conv7') as scope:
        conv7 = tf.layers.conv2d(inputs=pool6, filters=128, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv7',
                                kernel_regularizer = regularizer)
        weight7= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv7')[0]
        
        batch_norm7 = tf.layers.batch_normalization(conv7, axis = 1, name='batch_norm7')
        combine7 = tf.add(pool6, batch_norm7, name='combine7')
        dropout7 = tf.layers.dropout(combine7,rate=0.3, training=training, name='dropout7')
#         combine6 = tf.add(batch_norm5, batch_norm6, name='combine6')
#         dropout5 = tf.layers.dropout(batch_norm5,rate=0.3, training=training, name='dropout5')
#         pool6 = tf.layers.max_pooling2d(inputs=combine6, pool_size=2, strides=2, padding='SAME', name='pool6')
    
    with tf.name_scope('conv8') as scope:
        conv8 = tf.layers.conv2d(inputs=dropout7, filters=128, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv8',
                                kernel_regularizer = regularizer)
        weight8= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv8')[0]
        
        batch_norm8 = tf.layers.batch_normalization(conv8, axis = 1, name='batch_norm8')
        combine8 = tf.add(combine7, batch_norm8, name='combine8')
#         dropout8 = tf.layers.dropout(combine8,rate=0.4, training=training, name='dropout8')
    
    with tf.name_scope('fc') as scope:
        pool4_flat = tf.reshape(combine8, [-1, 4*4*128], name='pool4_flat')
        dropout_fc1 = tf.layers.dropout(pool4_flat, rate=0.4, training=training, name='dropout_fc1')
        
        dense1 = tf.layers.dense(dropout_fc1, units = 64, activation = tf.nn.relu, name='dense1',
                                kernel_regularizer = regularizer)
        batch_norm9 = tf.layers.batch_normalization(dense1, axis = 1, name='batch_norm9')
        dropout_fc2 = tf.layers.dropout(batch_norm9, rate=0.3, training=training, name='dropout_fc2')
        
#         dense2 = tf.layers.dense(dropout_fc2, units = 128, activation = tf.nn.relu, name='dense2',
#                                 kernel_regularizer = regularizer)
#         batch_norm6 = tf.layers.batch_normalization(dense2, axis = 1, name='batch_norm6')
#         dropout_fc3 = tf.layers.dropout(batch_norm6, rate=0.4, training=training, name='dropout_fc3')

    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(dropout_fc2, units = 10, name='logits',kernel_regularizer = regularizer)
        weight_logits = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'logits')[0]

    with tf.name_scope('loss') as scope:
        softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name = 'softmax')
        loss = tf.reduce_mean(softmax) 

    with tf.name_scope('train') as scope:
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    with tf.name_scope('eval') as scope:
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    
        
    with tf.name_scope('summary') as scope:
        save_variable(weight1,'weight1')
        save_variable(weight2,'weight2')
        save_variable(weight3,'weight3')
        save_variable(weight4,'weight4')
        save_variable(weight5,'weight5')
        save_variable(weight6,'weight6')
        save_variable(weight7,'weight7')
        save_variable(weight_logits,'weight_logits')
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(logdir + '/train', tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logdir + '/test', tf.get_default_graph())

    with tf.name_scope('init') as scope:
        init = tf.global_variables_initializer()
    

In [10]:
print(logits)

Tensor("logits/logits/BiasAdd:0", shape=(?, 10), dtype=float32, device=/device:GPU:0)


In [11]:
# config = tf.ConfigProto()
# config.allow_soft_placement = True
# with tf.Session(config=config) as sess:
    
#     X_batch, y_batch = get_batch(X_train, y_train, 1)
#     input_ ,distorted = sess.run([input_layer, distorted], feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
#     plt.imshow(input_)
#     plt.figure()
#     plt.imshow(distorted)

In [11]:
def train(X_train, y_train, X_validation, y_validation, iteration, X_test):

    config = tf.ConfigProto()
    config.allow_soft_placement = True
    
    with tf.Session(config=config) as sess:
        sess.run(init)
        for step in range(iteration):
            X_batch, y_batch = get_batch(X_train, y_train, batch_size)
            sess.run(optimizer, feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
                
            if step % 100 == 0:
                summary, loss_, acc = sess.run([merged, loss, accuracy],
                                               feed_dict={X: X_validation, y:y_validation, dropout_rate:0.3, training:False})
                test_writer.add_summary(summary, step)
                print('###################################')
                print('validation! after '+ str(step) + ' iterations' + 
                              ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
                        
                summary, loss_, acc = sess.run([merged, loss, accuracy], 
                                               feed_dict={X: X_batch, y:y_batch,dropout_rate:0.3, training:False})
                train_writer.add_summary(summary, step)
                        
                print('training! after '+ str(step) + ' iterations' + 
                          ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
            
        y_labels = logits.eval(feed_dict={X: X_test, dropout_rate:0.3, training:False})
    return y_labels.T

In [12]:
y_predicted = train(X_train, y_train, X_validation, y_validation, iteration, X_test)

###################################
validation! after 0 iterations the loss is 2.66599, the accuracy is 0.1034
training! after 0 iterations the loss is 2.48816, the accuracy is 0.125
###################################
validation! after 100 iterations the loss is 2.00286, the accuracy is 0.2492
training! after 100 iterations the loss is 1.9399, the accuracy is 0.21875
###################################
validation! after 200 iterations the loss is 1.75841, the accuracy is 0.3376
training! after 200 iterations the loss is 1.72918, the accuracy is 0.367188
###################################
validation! after 300 iterations the loss is 1.49964, the accuracy is 0.4498
training! after 300 iterations the loss is 1.4329, the accuracy is 0.5
###################################
validation! after 400 iterations the loss is 1.39848, the accuracy is 0.4818
training! after 400 iterations the loss is 1.29761, the accuracy is 0.5625
###################################
validation! after 500 iteration

###################################
validation! after 4300 iterations the loss is 0.583562, the accuracy is 0.8022
training! after 4300 iterations the loss is 0.746018, the accuracy is 0.742188
###################################
validation! after 4400 iterations the loss is 0.582473, the accuracy is 0.804
training! after 4400 iterations the loss is 0.310475, the accuracy is 0.882812
###################################
validation! after 4500 iterations the loss is 0.587391, the accuracy is 0.7974
training! after 4500 iterations the loss is 0.365527, the accuracy is 0.859375
###################################
validation! after 4600 iterations the loss is 0.560672, the accuracy is 0.8096
training! after 4600 iterations the loss is 0.389979, the accuracy is 0.875
###################################
validation! after 4700 iterations the loss is 0.572136, the accuracy is 0.8036
training! after 4700 iterations the loss is 0.552212, the accuracy is 0.8125
###################################


###################################
validation! after 8600 iterations the loss is 0.528443, the accuracy is 0.825
training! after 8600 iterations the loss is 0.226609, the accuracy is 0.929688
###################################
validation! after 8700 iterations the loss is 0.512786, the accuracy is 0.8328
training! after 8700 iterations the loss is 0.218503, the accuracy is 0.929688
###################################
validation! after 8800 iterations the loss is 0.504701, the accuracy is 0.8338
training! after 8800 iterations the loss is 0.187428, the accuracy is 0.9375
###################################
validation! after 8900 iterations the loss is 0.516479, the accuracy is 0.83
training! after 8900 iterations the loss is 0.196096, the accuracy is 0.929688
###################################
validation! after 9000 iterations the loss is 0.5094, the accuracy is 0.8348
training! after 9000 iterations the loss is 0.225042, the accuracy is 0.914062
###################################
v

###################################
validation! after 12900 iterations the loss is 0.483197, the accuracy is 0.856
training! after 12900 iterations the loss is 0.207867, the accuracy is 0.945312
###################################
validation! after 13000 iterations the loss is 0.470467, the accuracy is 0.855
training! after 13000 iterations the loss is 0.125443, the accuracy is 0.945312
###################################
validation! after 13100 iterations the loss is 0.480689, the accuracy is 0.85
training! after 13100 iterations the loss is 0.0905275, the accuracy is 0.96875
###################################
validation! after 13200 iterations the loss is 0.458323, the accuracy is 0.8542
training! after 13200 iterations the loss is 0.156229, the accuracy is 0.953125
###################################
validation! after 13300 iterations the loss is 0.489982, the accuracy is 0.847
training! after 13300 iterations the loss is 0.12175, the accuracy is 0.96875
###########################

In [13]:
save_predictions('ans2-yz3065', y_predicted)

In [14]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans2-yz3065.npy')
print(loaded_y.shape)
loaded_y[:10]

(10, 10000)


array([[ -0.07254168,  10.40071011,   6.96945763, ...,  -8.60781956,
         -0.30845132,  -5.30182266],
       [ -4.25405407,  12.66198921,  -6.03877735, ...,  -6.85159302,
         10.0753479 ,  -6.51480007],
       [ -0.51743031,  -1.4073509 ,   0.83241349, ...,   6.32799435,
         -0.39114469,  -0.57954508],
       ..., 
       [ -1.3303473 ,  -5.6273613 ,   0.79260683, ...,   5.71583748,
         -2.96765161,  23.36368942],
       [ -2.02939081,  23.23748589,  -2.87284589, ...,  -5.24417973,
         -2.71986508, -10.87773418],
       [ -3.29899573,   9.86066532,  -3.26939797, ...,  -7.88169718,
          3.98659897,  -4.21644115]], dtype=float32)